## Temporal Link Prediction in Co-Authorship Networks Using Graph Embedding Dynamics

Authors: Mohammad Ghassemi (ghassem3@msu.edu), Sanaz Hasanzadeh, Soroush Ziaeinejad

### Step 1: Import Required Libraries

In [3]:
# pip install dynamicgem
import csv
import DynamicGEM
import networkx as nx
from itertools import combinations
from dynamicgem.embedding.dynAERNN import DynAERNN

### Step 2: Import required Data and Extract Links

In [8]:
def Link_Extraction(path):
    
    data_1 = [] #each row corresponds to one paper, containing names of all authors of the paper (seperated by ";")
    
    with open(path, 'r', encoding="ISO-8859-1") as f:
        reader = csv.reader(f, delimiter=',')
        for row in reader:
            data_1.append(row[1])
            
        data_1[1:] #to remove the column name of dataset
        
    data_2 = [] #each row will be one paper's authors list, seperated by ","
    
    for i in range(len(data_1)):
        data_2.append(data_1[i].split('; '))

    authors_list = set() #each row is one author from data_2 list
    for i in range(len(data_2)):
        for j in range(len(data_2[i])):
            authors_list.add(data_2[i][j])
            
    #authors_list.remove('')
    
    authors_comb = [] #each row will be all possible combinations C(data_2[i], 2) based on who published with who (edges)
    for i in range(len(data_2)):
        authors_comb.append(list(combinations(data_2[i], 2)))

    # each row will be exactly one edge
    links = set()
    for i in range(len(authors_comb)):
        for j in range(len(authors_comb[i])):
            links.add(authors_comb[i][j])

    links = list(links) 
    
    return authors_list, links

    pass


#### Step3: Call "Link_Extraction" function on data

In [9]:
authors_list_2015, links_2015 = Link_Extraction('data/CSV/REPORTER_PUB_C_2015.csv')

authors_list_2016, links_2016 = Link_Extraction('data/CSV/REPORTER_PUB_C_2016.csv')

authors_list_2017, links_2017 = Link_Extraction('data/CSV/REPORTER_PUB_C_2017.csv')

authors_list_2018, links_2018 = Link_Extraction('data/CSV/REPORTER_PUB_C_2018.csv')

authors_list_2019, links_2019 = Link_Extraction('data/CSV/REPORTER_PUB_C_2019.csv')

#### 4- Intersection of all Years' Author-Set

In [9]:
authors_intersect = list(set.intersection(authors_list_2015, authors_list_2016, authors_list_2017, authors_list_2018, authors_list_2019))

#### 5- Generate Graphs

In [11]:
def Graph_Generate(links, authors_intersect):
    
    G = nx.Graph()
    all_links = filter(lambda c: c[0] in authors_intersect and c[1] in authors_intersect, links)
    list_all_links = list(all_links)
    #print("Num of Links: %float" %len(list_all_links))
    
    for i in range(len(list_all_links)):
        G.add_edge(list_all_links[i][0], list_all_links[i][1])
    return G

#### 6- Call Graph_Generate Function for each Year, Save Graphs

In [ ]:
G_2015 = Graph_Generate(links_2015, authors_intersect)
#print('2015 Graph is Generated')
G_2016 = Graph_Generate(links_2016, authors_intersect)
#print('2016 Graph is Generated')
G_2017 = Graph_Generate(links_2017, authors_intersect)
#print('2017 Graph is Generated')
G_2018 = Graph_Generate(links_2018, authors_intersect)
#print('2018 Graph is Generated')
G_2019 = Graph_Generate(links_2019, authors_intersect)
#print('2019 Graph is Generated')

nx.write_pajek(G_2015, "/data/Graphs/G-2015.net")
nx.write_pajek(G_2016, "/data/Graphs/G-2016.net")
nx.write_pajek(G_2017, "/data/Graphs/G-2017.net")
nx.write_pajek(G_2018, "/data/Graphs/G-2018.net")
nx.write_pajek(G_2019, "/data/Graphs/G-2019.net")

#### 7- Temporal Embedding
Output is a npy array: (4, 62641, 128)
first matrix is embeding for 2016 based on 2015
second one is embeding for 2017 based on 2016 and 2015
third one is embeding for 2018 based on 2017, 2016, and 2015
fourth one is embeding for 2019 based on 2018, 2017, 2016, and 2015

For any series of graphs the output will be in shape of (number of graphs-1, number of nodes, embedding dimension)

In [ ]:
import glob
import time
import numpy as np
import networkx as nx
import pickle5 as pickle
from dynamicgem.embedding.dynAERNN import DynAERNN

# Set Variables
dim_emb = 128
lookback = 1
batchNumber = 200
graphPath = '../data/Graphs/'

# To have sorted embedding in the Output
def node_sorter(graph):
    H = nx.Graph()
    new_nodes = sorted(graph.nodes(data=True))
    H.add_nodes_from(new_nodes)
    H.add_edges_from(graph.edges(data=True))
    return H

graphlist = sorted(glob.glob(f'{graphPath}/*.net'))
graphs = []
for i in graphlist:
    graphs.append(node_sorter(pickle.load(open(i, "rb"))))

length = len(graphs)
embedding = DynAERNN(d=dim_emb,
                     beta=5,
                     n_prev_graphs=lookback,
                     nu1=1e-6,
                     nu2=1e-6,
                     n_aeunits=[500, 300],
                     n_lstmunits=[500, dim_emb],
                     rho=0.3,
                     n_iter=25,
                     xeta=1e-3,
                     n_batch=batchNumber,
                     modelfile=['enc_model_dynAERNN.json',
                                'GEL/dec_model_dynAERNN.json'],
                     weightfile=['GEL/enc_weights_dynAERNN.hdf5',
                                 'GEL/dec_weights_dynAERNN.hdf5'],
                     savefilesuffix="testing")

embs = []
t1 = time.clock()
for temp_var in range(lookback + 1, length + 1):
    emb, _ = embedding.learn_embeddings(graphs[:temp_var])
    print('emb type: ', type(emb))
    embs.append(emb)
embs = np.asarray(embs)
e = embs.round(decimals=3)

print('embs shape: ', embs)
print(embedding._method_name + ':\n\tTraining time: %f' % (time.clock() - t1))
np.save(f'{graphPath}/embeddeds_dim{dim_emb}_batchNumber{batchNumber}_length{length}_lookback{lookback}.npy', embs)

### Working with Embedded Data

In [25]:
import glob
import random
import sklearn
import numpy as np
import networkx as nx
import pickle5 as pickle
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

### 1- Load embedded data

In [26]:
embeddings = np.load('data/EMBEDDING/embeddeds_dim128_batchNumber200_length5_lookback1.npy')

# We will predict 2019 collaborations based on 2018 and 2017 (these data is based on previous years, so prediction is based on
# all previous years 2015, 2016, 2017, and 2018)
embed2018 = embeddings[2]
embed2017 = embeddings[1]

In [27]:
# To have sorted embedding in Output
def node_sorter(graph):
    H = nx.Graph()
    new_nodes = sorted(graph.nodes(data=True))
    H.add_nodes_from(new_nodes)
    H.add_edges_from(graph.edges(data=True))
    return H

In [28]:
# nodes in all these graphs are their intersection, so nose_set is the same for all years

graph_2019 = nx.read_pajek('data/Graphs/pajek_graph19.net')
SortedNodeG2019 = node_sorter(graph_2019)
node_emb_dict_2018 = {list(SortedNodeG2019.nodes())[i] : embed2018[i] for i in range(len(SortedNodeG2019.nodes()))}
node_emb_dict_2017 = {list(SortedNodeG2019.nodes())[i] : embed2017[i] for i in range(len(SortedNodeG2019.nodes()))}

### 2- Generate Train/ Test dataset

In [8]:
# Number of negative samples is 4 times positive samples

num_pos_samp = len(SortedNodeG2019.edges())
num_neg_samp = 4*num_pos_samp

nodeList = list(SortedNodeG2019.nodes())

neg_set = []
while len(neg_set) < num_neg_samp:
    
    first = random.randint(0, len(SortedNodeG2019.nodes())-1)
    second = random.randint(0, len(SortedNodeG2019.nodes())-1)

    f = nodeList[first]
    s = nodeList[second]
                            
    if SortedNodeG2019.has_edge(f, s) == False:
        neg_set.append((f, s))
        
edgeList = list(SortedNodeG2019.edges())

# generate train/test data for baseline
pos_samp = []
for i in range(len(edgeList)):
    temp = []
    f = edgeList[i][0]
    s = edgeList[i][1]
    temp.append(f)
    temp.append(s)
    temp.append(node_emb_dict_2018[f])
    temp.append(node_emb_dict_2018[s])
    temp.append(1)
    pos_samp.append(temp)
    
neg_samp = []
for i in range(len(neg_set)):
    temp = []
    f = neg_set[i][0]
    s = neg_set[i][1]
    temp.append(f)
    temp.append(s)
    temp.append(node_emb_dict_2018[f])
    temp.append(node_emb_dict_2018[s])
    temp.append(-1)
    neg_samp.append(temp)   
    
# positive and negative samples in a unified list
samp = []
for i in pos_samp:
  samp.append(i)
for i in neg_samp:
  samp.append(i)    

shuf_samp = random.sample(samp, len(samp))
x = []
y = []
for i in range(len(shuf_samp)):
    x.append(np.concatenate((shuf_samp[i][2], shuf_samp[i][3]), axis=None))
    y.append(shuf_samp[i][4])
    
loc_xtrain, loc_xtest, loc_ytrain, loc_ytest = train_test_split(x, y, 
                                                test_size = 0.3, 
                                                random_state = 35)

In [7]:
# generate train/test data for velocity

vel_pos_samp = []
for i in range(len(edgeList)):
    temp = []
    f = edgeList[i][0]
    s = edgeList[i][1]
    temp.append(f)
    temp.append(s)
    temp.append(2*node_emb_dict_2018[f]-node_emb_dict_2017[f])
    temp.append(2*node_emb_dict_2018[s]-node_emb_dict_2017[s])
    temp.append(1)
    vel_pos_samp.append(temp)
               
vel_neg_samp = []
for i in range(len(neg_set)):
    temp = []
    f = neg_set[i][0]
    s = neg_set[i][1]
    temp.append(f)
    temp.append(s)
    temp.append(2*node_emb_dict_2018[f]-node_emb_dict_2017[f])
    temp.append(2*node_emb_dict_2018[s]-node_emb_dict_2017[s])
    temp.append(-1)
    vel_neg_samp.append(temp)  
    
vel_samp = []
for i in vel_pos_samp:
  vel_samp.append(i)
for i in vel_neg_samp:
  vel_samp.append(i)

vel_shuf_samp = random.sample(vel_samp, len(vel_samp))
vel_x = []
vel_y = []
for i in range(len(vel_shuf_samp)):
    vel_x.append(np.concatenate((vel_shuf_samp[i][2], vel_shuf_samp[i][3]), axis=None))    
    vel_y.append(vel_shuf_samp[i][4])
    
vel_xtrain, vel_xtest, vel_ytrain, vel_ytest = train_test_split(vel_x, vel_y, 
                                                test_size = 0.3, 
                                                random_state = 35)

### 3- Prediction

In [22]:
from sklearn.neural_network import MLPClassifier

modelb = MLPClassifier(alpha=1, max_iter=1000)
modelb.fit(loc_xtrain, loc_ytrain)
ybase = modelb.predict(loc_xtest)
modelv = MLPClassifier(alpha=1, max_iter=1000)
modelv.fit(vel_xtrain, vel_ytrain)
yvel = modelv.predict(vel_xtest)

print("baseline")
print(sklearn.metrics.roc_auc_score(loc_ytest, ybase))

print("velocity")
print(sklearn.metrics.roc_auc_score(vel_ytest, yvel))

baseline
0.6457894839470578
velocity
0.6963615936343718


In [8]:
embed2016 = embeddings[0]
node_emb_dict_2016 = {list(SortedNodeG2019.nodes())[i] : embed2016[i] for i in range(len(SortedNodeG2019.nodes()))}

# generate train/test data for acceleration

acc_pos_samp = []
for i in range(len(edgeList)):
    temp = []
    f = edgeList[i][0]
    s = edgeList[i][1]
    temp.append(f)
    temp.append(s)
    temp.append(2.25*node_emb_dict_2018[f]-1.5*node_emb_dict_2017[f]-0.25*node_emb_dict_2016[f])
    temp.append(2.25*node_emb_dict_2018[s]-1.5*node_emb_dict_2017[s]-0.25*node_emb_dict_2016[s])
    temp.append(1)
    acc_pos_samp.append(temp)
               
acc_neg_samp = []
for i in range(len(neg_set)):
    temp = []
    f = neg_set[i][0]
    s = neg_set[i][1]
    temp.append(f)
    temp.append(s)
    temp.append(2.25*node_emb_dict_2018[f]-1.5*node_emb_dict_2017[f]-0.25*node_emb_dict_2016[f])
    temp.append(2.25*node_emb_dict_2018[s]-1.5*node_emb_dict_2017[s]-0.25*node_emb_dict_2016[s])
    temp.append(-1)
    acc_neg_samp.append(temp)  
    
acc_samp = []
for i in acc_pos_samp:
  acc_samp.append(i)
for i in acc_neg_samp:
  acc_samp.append(i)

acc_shuf_samp = random.sample(acc_samp, len(acc_samp))
acc_x = []
acc_y = []
for i in range(len(acc_shuf_samp)):
    acc_x.append(np.concatenate((acc_shuf_samp[i][2], acc_shuf_samp[i][3]), axis=None))    
    acc_y.append(acc_shuf_samp[i][4])
    
acc_xtrain, acc_xtest, acc_ytrain, acc_ytest = train_test_split(acc_x, acc_y, 
                                                test_size = 0.3, 
                                                random_state = 35)

In [24]:
model_acc = MLPClassifier(alpha=1, max_iter=1000)
model_acc.fit(acc_xtrain, acc_ytrain)
y_acc = modelb.predict(acc_xtest)

print("acceleration")
print(sklearn.metrics.roc_auc_score(acc_ytest, y_acc))

acceleration
0.6711056226888197


In [1]:
from sklearn.metrics.pairwise import cosine_similarity

x_acc = []
y_acc = []
for i in range(len(edgeList)):
    f = edgeList[i][0]
    s = edgeList[i][1]
    x_acc.append(2.25*node_emb_dict_2018[f]-1.5*node_emb_dict_2017[f]-0.25*node_emb_dict_2016[f])
    y_acc.append(2.25*node_emb_dict_2018[s]-1.5*node_emb_dict_2017[s]-0.25*node_emb_dict_2016[s])
               
acc_neg_samp = []
for i in range(len(neg_set)):
    f = neg_set[i][0]
    s = neg_set[i][1]
    x_acc.append(2.25*node_emb_dict_2018[f]-1.5*node_emb_dict_2017[f]-0.25*node_emb_dict_2016[f])
    y_acc.append(2.25*node_emb_dict_2018[s]-1.5*node_emb_dict_2017[s]-0.25*node_emb_dict_2016[s]) 

acc_Similarities = cosine_similarity(x_acc, y_acc, dense_output=True)

NameError: name 'edgeList' is not defined

In [29]:
embed2016 = embeddings[0]
node_emb_dict_2016 = {list(SortedNodeG2019.nodes())[i] : embed2016[i] for i in range(len(SortedNodeG2019.nodes()))}

num_node = len(nodeList)

vel_loc_2019 = []
acc_loc_2019 = []

for i in range(num_node):
    node_name = nodeList[i]
    vel_loc_2019.append(2*node_emb_dict_2018[node_name]-node_emb_dict_2017[node_name])
    acc_loc_2019.append(2.25*node_emb_dict_2018[node_name]-1.5*node_emb_dict_2017[node_name]-0.25*node_emb_dict_2016[node_name])
 

In [33]:
import math
from sklearn.metrics.pairwise import cosine_similarity
import numpy

total_size = num_node
batch_size = 10
batch_num = math.ceil(total_size/batch_size)
start1 = 0
start2 = 0

acc_cosine_similarity = [[0 for x in range(num_node)] for y in range(num_node)] 

0


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [49]:
import numpy
from sklearn.metrics.pairwise import cosine_similarity

acc_loc_2019 = np.asarray(acc_loc_2019, dtype=np.float32)

start1 = 0
start2 = 0

for i in range(batch_num):
    for j in range(batch_num):
        acc_cosine_similarity[start1: start1 + batch_size][start2: start2 + batch_size] = \
        cosine_similarity(acc_loc_2019[start1: start1 + batch_size], \
                                                   acc_loc_2019[start2: start2 + batch_size], \
                          dense_output=True)
        start2 = (j+1)*batch_size
    start1 = (i+1)*batch_size
    start2 = 0
    if(i%10 == 0):
        print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [50]:
acc_cosine_sim_file = open("acc_sim.txt", "w")
for row in acc_cosine_similarity:
    np.savetxt(acc_cosine_sim_file, row)

acc_cosine_sim_file.close()

KeyboardInterrupt: 

In [81]:
total_size = num_node
batch_size = 10
batch_num = math.ceil(total_size/batch_size)
start = 0

acc_loc_2019 = np.array(acc_loc_2019)
batches = []
for i in range(batch_num):
    batches.append(acc_loc_2019[start: start+batch_size])
    start = start+batch_size

In [87]:
round_batches = batches[:len(batches)-1]
all_acc_sim = []
count = 0
for i in round_batches:
    for j in round_batches:
        all_acc_sim.append(cosine_similarity(i, j, dense_output=True))
    if count%10 == 0:
        print(count)
    count = count + 1

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [94]:
all_acc_sim[0][1]

array([ 0.36765906,  0.99999964, -0.05642955,  0.05130933,  0.10695032,
        0.07395042,  0.2655197 ,  0.1603888 ,  0.08726624,  0.17522533],
      dtype=float32)